In [1]:
import os, mlflow
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier

In [2]:
# print(os.getcwd())

In [3]:
# os.chdir(".\desktop\batch1")
# print(os.getcwd())

In [4]:
df = pd.read_csv(R"C:\Users\suhaimi\Desktop\batch1\datasets\titanic\train.csv")

In [5]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
# EDA
# Find the value count of Name
df["Name"].value_counts()

Name
Braund, Mr. Owen Harris                     1
Boulos, Mr. Hanna                           1
Frolicher-Stehli, Mr. Maxmillian            1
Gilinski, Mr. Eliezer                       1
Murdlin, Mr. Joseph                         1
                                           ..
Kelly, Miss. Anna Katherine "Annie Kate"    1
McCoy, Mr. Bernard                          1
Johnson, Mr. William Cahoone Jr             1
Keane, Miss. Nora A                         1
Dooley, Mr. Patrick                         1
Name: count, Length: 891, dtype: int64

In [7]:
df = df.drop(["Name", "PassengerId", "Ticket"], axis=1)
df = df.drop(["Cabin"], axis=1)

In [8]:
print(df.isna().sum())

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64


In [9]:
# df = df['Embarked'].fillna(value=df['Embarked'].mode()[0])
df=df.fillna({'Embarked':df['Embarked'].mode()[0]})
print(df.isna().sum())  

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      0
dtype: int64


In [10]:
# MISSING THIS CHUNK OF CODE. YOU FORGOT TO FILL IN THE VALUE FOR EMPTY AGE GROUP. 
df = df.fillna({'Age':0.5})
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [11]:
age_range = [-1, 0, 5, 12, 18, 24, 35, 60, np.inf]
age_group = ['Unknown', 'Baby', 'Child', 'Teenage', 'Student', 'Yound Adult', 'Adult', 'Senior']
#df['AgeGroup'] = pd.cut(age_range, age_group)
df['AgeGroup'] = pd.cut(df['Age'], labels=age_group,bins=age_range)

In [12]:
ordinal_encoder_2 = preprocessing.OrdinalEncoder()
df[['Sex', 'Embarked']] = ordinal_encoder_2.fit_transform(df[['Sex', 'Embarked']].values)
print(df[['Sex', 'Embarked']])

     Sex  Embarked
0    1.0       2.0
1    0.0       0.0
2    0.0       2.0
3    0.0       2.0
4    1.0       2.0
..   ...       ...
886  1.0       2.0
887  0.0       2.0
888  0.0       2.0
889  1.0       0.0
890  1.0       1.0

[891 rows x 2 columns]


In [13]:
print(df['Sex'].unique())
print(df['Embarked'].unique())
print(df['AgeGroup'].unique())

[1. 0.]
[2. 0. 1.]
['Student', 'Adult', 'Yound Adult', 'Baby', 'Teenage', 'Child', 'Senior']
Categories (8, object): ['Unknown' < 'Baby' < 'Child' < 'Teenage' < 'Student' < 'Yound Adult' < 'Adult' < 'Senior']


In [14]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup
0,0,3,1.0,22.0,1,0,7.2500,2.0,Student
1,1,1,0.0,38.0,1,0,71.2833,0.0,Adult
2,1,3,0.0,26.0,0,0,7.9250,2.0,Yound Adult
3,1,1,0.0,35.0,1,0,53.1000,2.0,Yound Adult
4,0,3,1.0,35.0,0,0,8.0500,2.0,Yound Adult


In [15]:
ordinal_encode = preprocessing.OrdinalEncoder(categories=[age_group])
df['AgeGroup'] = ordinal_encode.fit_transform(df['AgeGroup'].values.reshape(-1, 1))

In [22]:
features = df.copy()
label = features.pop('Survived')
x_train, x_test, y_train, y_test = model_selection.train_test_split(features, label, train_size=0.8, shuffle=True, random_state=42)

In [25]:
# Feature scaling
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('scaler', preprocessing.StandardScaler()),
    ('model', RandomForestClassifier())
])

In [18]:
# Conduct the training  in mfow experiment
#mlflow.create_experiment('titanic_experiment')
mlflow.set_experiment('titanic_experiment')
mlflow.set_experiment_tag('ML_Flow_Version', '2.15.1')    

In [19]:
# os.chdir(r"C:\Users\suhaimi\Desktop\batch1")
# print(os.getcwd())
print(df["Sex"].unique())

[1. 0.]


In [27]:
with mlflow.start_run():
    mlflow.sklearn.autolog()
    pipeline.fit(x_train, y_train.values)
    predictions = pipeline.predict(x_test)
    accuracy = metrics.accuracy_score(y_test.values, predictions)
    precision = metrics.precision_score(y_test.values, predictions)
    recall = metrics.recall_score(y_test.values, predictions)
    f1 = metrics.f1_score(y_test.values, predictions)

2024/08/08 14:26:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\suhaimi\AppData\Local\anaconda3\envs\ciast_ai01\Lib\site-packages\mlflow\types\utils.py:406: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2024/08/08 14:26:11 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\suhaimi\AppDat

In [28]:
import pickle as pkl
help(pkl.dump)

Help on built-in function dump in module _pickle:

dump(obj, file, protocol=None, *, fix_imports=True, buffer_callback=None)
    Write a pickled representation of obj to the open file object file.

    This is equivalent to ``Pickler(file, protocol).dump(obj)``, but may
    be more efficient.

    The optional *protocol* argument tells the pickler to use the given
    protocol; supported protocols are 0, 1, 2, 3, 4 and 5.  The default
    protocol is 4. It was introduced in Python 3.4, and is incompatible
    with previous versions.

    Specifying a negative protocol version selects the highest protocol
    version supported.  The higher the protocol used, the more recent the
    version of Python needed to read the pickle produced.

    The *file* argument must have a write() method that accepts a single
    bytes argument.  It can thus be a file object opened for binary
    writing, an io.BytesIO instance, or any other custom object that meets
    this interface.

    If *fix_import

In [35]:
print(os.getcwd())
# os.chdir("..")

c:\Users\suhaimi\Desktop\batch1


In [40]:
with open("src/ordinal_enconder_1.pkl", "wb") as f:
    pkl.dump(ordinal_encode, f)

with open("src/ordinal_enconder_2.pkl", "wb") as f:
    pkl.dump(ordinal_encoder_2, f)